In [1]:
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import pandas as pd
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime
import pandas as pd

# Clients inputs example for using the model

In [2]:
# Client loads the model and scalar
gru_model=load_model('gru_model.h5')
gru_scalar=joblib.load('gru_scalar.pkl')
# Get last 6 days daya to return predictions on the next week
gru_exmple = {'Date':['2023-09-12', '2023-09-13', '2023-09-14', '2023-09-15', '2023-09-18', '2023-09-19'],
            'Close': [444.422363, 444.940552, 448.777008, 443.369995, 443.630005, 442.709991]}
# Get next week predictions
num_days_to_predict = 7

# Auxiliary functions

In [3]:
def create_next_predictions(model, scalar, generator, num_days_to_predict, length, batch_size):
    """
    Generates predictions for the next `num_days_to_predict` days using the specified model and data generator.

    Parameters:
    - model: the trained model used for making predictions
    - scalar: the scalar used to scale the data
    - generator: the data generator used to generate the next days' sequences
    - num_days_to_predict: the number of days to generate predictions for
    - length: the length of the input sequence used for prediction
    - batch_size: the batch size used for generating the sequences

    Returns:
    - next_days_to_predict: a list of next days' predictions
    """
    next_days_to_predict = []

    # Generate predictions for each day
    for day in range(num_days_to_predict):
        prev_days, next_day_to_predict = generator[0]

        # Flatten the input sequences
        prev_days = prev_days.flatten()
        next_day_to_predict = next_day_to_predict.flatten()

        # Concatenate the previous days, next day to predict, and model predictions
        next_days_sequents = list(prev_days[1:]) + list(next_day_to_predict) + list(model.predict(generator)[0])
        next_days_sequents = np.array(next_days_sequents)

        # Reshape the data to match the input shape of the model
        next_days_sequents = next_days_sequents.reshape((6, 1))

        # Create a new generator with the next days' sequences
        generator = TimeseriesGenerator(next_days_sequents, next_days_sequents, length=length, batch_size=batch_size)

        # Inverse transform the predictions using the scalar
        next_days_to_predict += [scalar.inverse_transform(model.predict(generator))]

    return next_days_to_predict

In [4]:
def create_generator(sample_json):
    """
    Creates a data generator using the given sample JSON data.

    Parameters:
    - sample_json: the sample JSON data to create the generator from

    Returns:
    - generator: the created data generator
    """
    gru_example = pd.DataFrame(data=sample_json)

    # Convert the 'Date' column to datetime format
    gru_example['Date'] = pd.to_datetime(gru_example['Date'])

    # Set the 'Date' column as the index
    gru_example.set_index('Date', inplace=True)

    length = 5
    batch_size = 1

    # Scale the data using the 'gru_scalar' scalar
    scaled_gru_example = gru_scalar.transform(gru_example)

    # Create a time series generator using the scaled data
    generator = TimeseriesGenerator(scaled_gru_example, scaled_gru_example, length=length, batch_size=batch_size)

    return generator

In [5]:
def create_next_days_predictions_df(next_days_predictions, starting_date):
    """
    Creates a DataFrame with the next days' predictions and corresponding dates.

    Parameters:
    - next_days_predictions: a list of next days' predictions
    - starting_date: the starting date for the predictions

    Returns:
    - next_days_predictions_df: the created DataFrame
    """
    # Create an empty dataframe
    next_days_predictions_df = pd.DataFrame()

    # Create the "predictions" column with values from the numpy arrays
    next_days_predictions_df['predictions'] = [arr.item() for arr in next_days_predictions]

    # Convert starting_date to datetime object
    starting_date = datetime.strptime(starting_date, '%Y-%m-%d')

    # Create the "date" column with incremented dates
    next_days_predictions_df['date'] = pd.date_range(start=starting_date + pd.DateOffset(days=1), periods=len(next_days_predictions), freq='D')

    return next_days_predictions_df    

In [6]:
# This is the function that the client uses
def return_prediction(model, scalar, sample_json, num_days_to_predict):
    """
    Returns the predictions for the next `num_days_to_predict` days.

    Parameters:
    - model: the trained model used for making predictions
    - scalar: the scalar used to scale the data
    - sample_json: the sample JSON data to generate predictions from
    - num_days_to_predict: the number of days to generate predictions for

    Returns:
    - next_days_predictions_df: a DataFrame containing the predictions for the next days
    """
    last_date_example = sample_json['Date'][-1]

    # Create a data generator from the sample JSON data
    generator = create_generator(sample_json)

    # Generate predictions for the next days using the model and generator
    next_days_predictions = create_next_predictions(model=model, 
                                                    scalar=scalar, 
                                                    generator=generator, 
                                                    num_days_to_predict=num_days_to_predict, 
                                                    length=5, 
                                                    batch_size=1)
    
    # Create a DataFrame with the predicted values and dates
    next_days_predictions_df = create_next_days_predictions_df(next_days_predictions, last_date_example)
    
    return next_days_predictions_df


# Clients function to retrieve next week predictions

In [7]:
return_prediction(model=gru_model, scalar=gru_scalar, sample_json=gru_exmple, num_days_to_predict=num_days_to_predict)

1/1 [==============================] - 0s 29ms/step


,predictions,date
0,445.069733,2023-09-20
1,448.518555,2023-09-21
2,447.161102,2023-09-22
3,450.771149,2023-09-23
4,449.181488,2023-09-24
5,452.997894,2023-09-25
6,451.144287,2023-09-26
